<a href="https://colab.research.google.com/github/usamireko/WFL-ASR/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install and Preparation
from google.colab import drive
from IPython.display import clear_output, display, Javascript
drive.mount('/content/drive')
message = """
alert("Installing components for inference");
"""
!git clone https://github.com/MLo7Ghinsan/WFL-ASR
!wget https://github.com/MLo7Ghinsan/WFL-ASR/releases/download/model_release/model_1.0.zip
!unzip model_1.0.zip -d /content/model
%cd /content/WFL-ASR
display(Javascript(message))
!pip install -r requirements.txt
clear_output()


In [ ]:
#@title Apply settings
%%writefile /content/model/config.yaml

data:
  data_dir: /content/Training_dataset
  sample_rate: 16000
  num_val_files: 10
  max_seq_len: null
model:
  encoder_type: whisper
  whisper_model: openai/whisper-base
  wavlm_model: microsoft/wavlm-base
  freeze_encoder: false
  enable_bilstm: true
  bilstm_num_layer: 2
  enable_dilated_conv: true
  dilated_conv_depth: 2
  dilated_conv_kernel: 3
  enable_duration_prediction: true
  duration_head_dim: 128
  duration_loss_weight: 0.2
  enable_self_attn_polisher: false
  self_attn_heads: 2
  num_conformer_layers: 2
  conformer_heads: 2
  conformer_ff_expansion: 2
  conformer_kernel_size: 31
  conformer_dropout: 0.5
  lang_emb_dim: 64
  num_languages: 2
training:
  batch_size: 1
  num_workers: 4
  learning_rate: 0.0000001
  weight_decay: 1.0e-05
  label_smoothing: 0.1
  max_steps: 500000
  val_check_interval: 1000
  max_checkpoints: 5
  log_dir: /content/logs
output:
  save_dir: /content/model
postprocess:
  median_filter: 2
  merge_segments: previous


English data == 0 as lang_id
Japanese data == 1 as lang_id

In [ ]:
#@title # Single Audio Inference
import os
audio_file = "" # @param {"type":"string"}
lang_id = "1" # @param ["0","1"]
name = os.path.splitext(os.path.basename(audio_file))[0]
output_lab = "/content/drive/MyDrive/data/" + name + ".lab"
!python infer.py {audio_file} /content/model/best_model.pt /content/model/config.yaml {output_lab} --lang_id {lang_id}
clear_output()
print("Labels generated, check them under " + output_lab)

In [ ]:
#@title # Folder Inference
folder_path = "" # @param {"type":"string"}
lang_id = "1" # @param ["0","1"]
output_path = folder_path + "/labs"
!python infer.py --folder {folder_path} /content/model/best_model.pt /content/model/config.yaml {output_path}  --lang_id {lang_id}
clear_output()
print("Labels generated, check them under " + output_path)